In [ ]:
import codecs
import requests
from bs4 import BeautifulSoup
import random
from collections import OrderedDict
import re
from types import *

headers_of_request = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.87 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, sdch',
    'Accept-Language': 'en-US,en;q=0.5',
    'Connection': 'keep-alive',
    'Host': 'www.wango.org'
}

def relevant_links(tag):
    a = False
    if (tag.name==u'a' and tag.has_attr('href')):
        if (tag['href'].find('javascript:loadOrg(')!=-1):
            a = True
    return a

s = requests.Session()
s.headers.update(headers_of_request)

.. code:: python:

    def relevant_links(tag):
        a = false
        if (tag.name==u'a' and tag.has_attr('href')):
            if (tag['href'].find('javascript:loadOrg(')!=-1):
                ngo_id = re.search("javascript:loadOrg((.+?));",tag['href'])
                if ngo_id:
                    a = ngo_id.group(1)[2:-2]
            else:
                a = '-1'
        else:
            a = '-1'
        return int(float(a))

In [ ]:
BaseUrl = 'http://www.wango.org'
SubUrl = '/resources.aspx?section=ngodir'

'''SubSubUrl = '&sub=region&regionID=5&col=cc3300'
'''

payload = OrderedDict([('sub','region'),('regionID', '5'),('col', 'cc3300')])



ngo_initial_response = s.get(url=(BaseUrl + SubUrl),params=payload)
print ngo_initial_response.url
print ngo_initial_response.status_code
print ngo_initial_response.request.body

In [ ]:
request_item = OrderedDict([
    ("Countries", "32,68,76,152,170,218,238,254,328,600,604,740,858,862,0"),
    ("InterestAreas", "Environment"),
    ("currpage", 1)
])
referer = {
    'Referer': ngo_initial_response.url
}



payload2 = OrderedDict([('sub','list'),('regionID', '5'),('col', 'cc3300')])
ngo_filtered_response = s.post(url=(BaseUrl + SubUrl),params=payload2, data=request_item, headers=referer)
print ngo_filtered_response.url
print ngo_filtered_response.status_code
print ngo_filtered_response.request.body

In [ ]:
ngo_soup = BeautifulSoup(ngo_filtered_response.content,'lxml')

print ngo_soup.prettify()

In [ ]:
filtered_content = ngo_soup.find('div',{'class': 'content'})
filtered_pages_content = filtered_content.find('td',{'valign': 'top'}).find('div',{'align': 'left'})
number_of_pages = -1

for link in filtered_pages_content.findAll('a'):
    number_of_pages += 1
    


In [ ]:
main_content = filtered_content.find('td',{'valign': 'top'})
print main_content.prettify()

In [ ]:
for link in main_content.findAll(relevant_links):
    ngo_id = int(float(re.search("javascript:loadOrg((.+?));",
                                 link['href']).group(1)[2:-2]))
    
   